In [1]:
import tweepy
import configparser
import time
import json

In [2]:
# keys
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [3]:
c_key = config['twitter']['C_KEY']
c_secret = config['twitter']['C_SECRET']
a_token = config['twitter']['A_TOKEN']
a_token_secret = config['twitter']['A_TOKEN_SECRET']

bearer_token = config['twitter']['bearer_token']

# authenticate
client = tweepy.Client(bearer_token= bearer_token,consumer_key = c_key, consumer_secret = c_secret, access_token = a_token, access_token_secret= a_token_secret, wait_on_rate_limit=True)

# auth = tweepy.OAuthHandler(api_key, api_key_secret)
# auth.set_access_token(access_token, access_token_secret)
# api = tweepy.API(auth)

In [4]:

# Create a StreamListener class 
class MyStreamingClient(tweepy.StreamingClient):

    def __init__(self,bearer_token, time_limit = 300):
        self.start_time = time.time()
        self.limit = time_limit
        self.outFile = open('output.json', 'a', encoding = "utf-8")
        super(MyStreamingClient, self).__init__(bearer_token)
    
    def on_data(self, data):
        #if (time.time() - self.start_time) < self.limit:
        self.outFile.write(data.decode('utf-8'))
        self.outFile.write('\n')
        #else:
        if (time.time() - self.start_time) > self.limit:
            self.outFile.close()
            self.disconnect()
            
    def on_error(self, status):
        print(status)

In [5]:
streaming_client = MyStreamingClient(bearer_token)
streaming_client.add_rules(tweepy.StreamRule("michigan"))
streaming_client.filter()

Stream connection closed by Twitter


In [6]:
streaming_client.get_rules()

Response(data=[StreamRule(value='michigan', tag=None, id='1580724917594636288')], includes={}, errors=[], meta={'sent': '2022-10-15T01:05:06.901Z', 'result_count': 1})

In [7]:
import json

f = open('output.json',encoding= 'utf-8')
tweets = [json.loads(line) for line in f]
f.close()

In [8]:
hashtag_list = []

from collections import defaultdict

counts = defaultdict(int)   # count values are initialized to 0


# splitting the text into words
for tweet in tweets:
    words = tweet['data']['text'].split()   
    # checking the first character of every word
    for word in words:
#         print(word[0])
        if word[0] == '#':
        # adding the word to the hashtag_list
            counts[word] += 1
            hashtag_list.append(word[1:])

In [9]:
import operator
sorted_words = sorted(counts.items(), key=operator.itemgetter(1))
sorted_words[-5:]

[('#Democrat', 2),
 ('#FollowTheHuskies', 2),
 ('#Michigan', 4),
 ('#GoRedStateByState', 4),
 ('#GoBlue', 4)]